# **Optimal Preconditioner Problem**

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.sparse.linalg import gmres
import scipy.sparse as sp
from scipy.linalg import svd

# ToDos / Ideas
- [ ] Usar un método de optimización que use gradiente y pre calcular el gradiente analitico seguramente va a disminuir el tiempo de computo del precondicionador.
- [ ] Ide de en lugar de intentar llegar a una identidad usar pesos y llegar a una triangular inferior

## **Funciones Auxiliares**

In [ ]:
def debug(msg,verbose=True):
	if verbose:
		print(msg)
	else:
		pass


def is_valid_M_structure(config):
	valid_structure = np.array(["k-diagonal", "k-triangular-low"])
	return 'M-structure' in config and np.isin(config['M-structure'], valid_structure)

def is_valid_A_structure(config):
	valid_structure = np.array(["default", "k-triangular-low", "k-triangular-up","k-diagonal"])
	return 'A-structure' in config and np.isin(config['A-structure'], valid_structure)

def is_valid_multiplication_mode(config):
	valid_modes = np.array(["left-multiplication", "both-sides-multiplication"])
	return 'multiplication-mode' in config and np.isin(config['multiplication-mode'], valid_modes)


def num_elements_M(n,k,structure):
	if structure == "k-triangular-low":
		sum = (k)*n
		res = (k*(k-1))/2
		return sum-res
	elif structure == "k-diagonal":
		sum = 2*n*(k-1) + n
		res = (k-1)*(k)
		return sum-res
	

def is_valid_values(n,k,len,structure):
	expected_elements = num_elements_M(n,k,structure)
	if structure == "k-triangular-low":
		valid =  expected_elements == len and k>0 and k<n and n>=3
		return valid
	elif structure == "k-diagonal":
		valid = expected_elements == len and k>0 and k<n and n>=3
		return valid


def validate_M_config(M_config):
	if not is_valid_M_structure(M_config):
		raise TypeError("Error en la variable config['M-structure']")

	if 'n' not in M_config or not isinstance(M_config['n'], int):
		raise TypeError("El 'config' debe contener 'n' como un entero (int).")

	if 'k' not in M_config or not isinstance(M_config['k'], int):
		raise TypeError("El 'config' debe contener 'k' como un entero (int).")

	if not is_valid_multiplication_mode(M_config):
		raise TypeError("Error en la variable config["multiplication-mode"]")
	
	if 'seed' not in M_config or not isinstance(M_config['seed'], int):
		raise TypeError("El 'config' debe contener 'seed' como un entero (int).")
	
	if 'optimization-mode' not in M_config or not isinstance(M_config['optimization-mode'], str):
		raise TypeError("El 'config' debe contener 'optimization-mode' como un string (str).")
	
	if 'tol' not in M_config or not isinstance(M_config['tol'], float):
		raise TypeError("El 'config' debe contener 'tol' como un float (float).")	
	

def validate_A_config(A_config):
	#k is optional
	if not is_valid_A_structure(A_config):
		raise TypeError("Error en la variable config['A-structure']")

	if 'n' not in A_config or not isinstance(A_config['n'], int):
		raise TypeError("El 'config' debe contener 'n' como un entero (int).")

	if 'seed' not in A_config or not isinstance(A_config['seed'], int):
		raise TypeError("El 'config' debe contener 'seed' como un entero (int).")
	
	if 'sparsity' not in A_config or not isinstance(A_config['sparsity'], float):
		raise TypeError("El 'config' debe contener 'sparsity' como un float (float).")	

	if 'diagonally-dominant' not in A_config or not isinstance(A_config['diagonally-dominant'], bool):
		raise TypeError("El 'config' debe contener 'diagonally-dominant' como un bool (bool).")	

## **Contrucción matrices $M$, $A$**

In [ ]:
def build_M(M_values, M_config):
	structure = M_config["M-structure"]
	n = M_config["n"]
	k = M_config["k"]
	# ::===========================  SANITY CHECKS ===========================::
	if not is_valid_values(n,k,len(M_values),structure):
		raise ValueError("La matriz de valores tiene un problema, ")
	debug("Verificaciones de configuración completadas con éxito")
	# ::===========================  SANITY CHECKS ===========================::


	# ::======================  CASE: k-triangular-low  ======================::
	M = np.zeros((n, n))
	if structure == "k-triangular-low":
		values_temp = M_values.copy()
		start = 0
		for offset in range(k):
			length = n - offset
			diag = values_temp[start:start + length]
			start += length
			np.fill_diagonal(M[offset:, :n - offset], diag)
	# ::======================  CASE: k-triangular-low  ======================::

	# ::=========================  CASE: k-diagonal  =========================::
	elif structure == "k-diagonal":
		values_temp = M_values.copy()
		diagonals = list(range(k-1, -k, -1))   
		start = 0 
		for k in diagonals:
			length = n - abs(k)
			if start >= len(values_temp) or length <= 0:
				break
			end = start + length
			diag_values = values_temp[start:end]
			start = end
			if k >= 0:
				rows = np.arange(length)
				cols = np.arange(k, k + length)
			else:
				# Diagonales inferiores
				rows = np.arange(-k, -k + length)
				cols = np.arange(length)

			M[rows, cols] = diag_values
	# ::=========================  CASE: k-diagonal  =========================::
	
	return M

In [ ]:
# A_config = {
#     "n":5,
#     "k":3,
#     "seed":67,
#     "cond_number":100,
#     "A-structure": "default",
#     "sparsity":0.0,
#     "diagonally-dominant": True
# }

def build_A(A_config):
# def generate_test_matrix(n, seed=None, cond_num=None, diagonal_dominant=False, 
                        # structure='dense', k=1, triangular_type='lower'):
    seed = A_config["seed"]
    n = A_config["n"]
    if seed is not None:
        np.random.seed(seed)
    
    # Generar matriz base según la estructura
    if structure == 'dense':
        A = np.random.randn(n, n)
        
    elif structure == 'sparse':
        # Matriz sparse con aproximadamente 10% de elementos no cero
        density = 0.1
        A = sp.random(n, n, density=density, random_state=seed).toarray()
        A = A + np.eye(n)  # Añadir diagonal para asegurar invertibilidad
        
    elif structure == 'k-diagonal':
        A = np.zeros((n, n))
        # k-diagonal: k especifica el número total de diagonales
        # k=1: diagonal, k=2: tridiagonal, k=3: pentadiagonal, etc.
        bandwidth = (k - 1) // 2
        for i in range(-bandwidth, bandwidth + 1):
            diag_length = n - abs(i)
            if diag_length > 0:
                diag_vals = np.random.randn(diag_length)
                np.fill_diagonal(A[i:], diag_vals)
                
    elif structure == 'k-triangular':
        A = np.zeros((n, n))
        if triangular_type == 'lower':
            for i in range(k):  # i=0 (diagonal), i=1 (subdiagonal 1), etc.
                diag_length = n - i
                if diag_length > 0:
                    diag_vals = np.random.randn(diag_length)
                    np.fill_diagonal(A[i:], diag_vals)
        else:  # upper
            for i in range(k):
                diag_length = n - i
                if diag_length > 0:
                    diag_vals = np.random.randn(diag_length)
                    np.fill_diagonal(A[:, i:], diag_vals)
    else:
        raise ValueError(f"Estructura no soportada: {structure}")
    
    # Hacer la matriz diagonal dominante si se solicita
    if diagonal_dominant:
        row_sums = np.sum(np.abs(A), axis=1)
        np.fill_diagonal(A, row_sums + np.random.rand(n) + 0.1)
    
    # Ajustar número de condición si se especifica
    if cond_num is not None and cond_num > 1:
        A = adjust_condition_number(A, cond_num)
    
    return A

def adjust_condition_number(A, cond_num):
    """
    Ajusta el número de condición de una matriz.
    
    Parameters:
    -----------
    A : ndarray
        Matriz de entrada
    cond_num : float
        Número de condición deseado
        
    Returns:
    --------
    A_cond : ndarray
        Matriz con el número de condición especificado
    """
    U, s, Vt = svd(A)
    
    # Ajustar valores singulares
    s_min = s[-1]
    s_max = s[0]
    
    if s_max / s_min < cond_num:
        # Escalar para alcanzar el número de condición deseado
        s_new = np.linspace(s_max, s_max / cond_num, len(s))
    else:
        # Preservar la forma pero ajustar el rango
        s_new = (s - s_min) / (s_max - s_min) * (s_max - s_max/cond_num) + s_max/cond_num
    
    # Reconstruir la matriz
    A_cond = U @ np.diag(s_new) @ Vt
    return A_cond

# Función auxiliar para verificar propiedades
def analyze_matrix(A, name="Matriz"):
    """Analiza propiedades importantes de una matriz."""
    print(f"\n{name}:")
    print(f"  Forma: {A.shape}")
    print(f"  Número de condición: {np.linalg.cond(A):.2e}")
    print(f"  Rango: {np.linalg.matrix_rank(A)}")
    print(f"  ¿Diagonal dominante?: {is_diagonally_dominant(A)}")
    print(f"  Norma Frobenius: {np.linalg.norm(A, 'fro'):.2f}")

def is_diagonally_dominant(A):
    """Verifica si una matriz es diagonal dominante."""
    diag = np.diag(np.abs(A))
    off_diag_sum = np.sum(np.abs(A), axis=1) - diag
    return np.all(diag > off_diag_sum)

## **Función Objetivo**

In [ ]:
def objective_fun(M_values, A, M_config):
	n = M_config["n"]
	M = build_M(M_values,M_config)
	multiplication_mode = M_config["multiplication-mode"]
		
	if multiplication_mode == "left-multiplication":
		diff = M @ A - np.eye(n)
	elif multiplication_mode == "both-sides-multiplication":
		diff = M @ A @ M.T - np.eye(n)
				
	return np.linalg.norm(diff, 'fro')

In [ ]:
def optimizer(A, M_config, x0, objective_fun):
	res = minimize(
		fun=objective_fun,
		x0=x0,
		args=(A,M_config), 
		method=M_config["optimization-mode"],
		tol=M_config["tol"]
	)        
	return res.x

In [ ]:
M_config = {
	"M-structure":"k-triangular-low",
	"multiplication-mode":"both-sides-multiplication",
	"optimization-mode":"BFGS",
	"seed":67,
	"n":5,
	"k":3,
	"tol":1e-8
}

A_config = {
    "n":5,
    "k":3,
    "seed":67,
    "cond_number":100,
    "A-structure": "default",
    "sparsity":0.0,
    "diagonally-dominant": True
}

